In [3]:
import os
import concurrent.futures

from tqdm import tqdm
from openai import OpenAI
from typing import List, Dict, Optional

prefix = "Below is a question followed by some context from different sources. Please answer the question based on the context. The answer to the question is a word or entity. If the provided information is insufficient to answer the question, respond 'Insufficient Information'. Answer directly without explanation."

def process_qa_pipeline(
    input_path: str,
    output_path: str,
    model_name: str,
    api_key: str,
    api_url: str,
    workers: int = 100,
    
) -> List[Dict]:
    """
    完整的QA处理流水线
    参数:
    input_path: 输入文件路径（JSON格式）
    output_path: 输出文件路径（JSON格式）
    
    返回:
    处理结果列表，每个元素包含：
        - query: 原问题
        - prompt: 生成的完整提示
        - model_answer: 模型回答
        - gold_answer: 正确答案
        - question_type: 问题类型
    """
    with open(input_path, 'r') as f:
        process_data = json.load(f)

    # 初始化OpenAI客户端
    client = OpenAI(api_key=api_key, base_url=api_url)

    def query_model(prompt: str) -> str:
        
        """模型调用统一入口"""
        if "gpt" in model_name.lower() or model_name == "deepseek-reasoner":
            try:
                response = client.chat.completions.create(
                    model=model_name,  # 使用传入的模型名称参数
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0
                )
                return response.choices[0].message.content.strip()
            except Exception as e:
                print(f"API调用失败: {str(e)}")
                return "Error: API调用失败"
            
        if model_name == "qwq-plus":      
            answer_content = ""     # 定义完整回复
            reasoning_content = ""
            is_answering = False
            completion = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
                stream=True,
            )
            for chunk in completion:
                # 如果chunk.choices为空，则打印usage
                if not chunk.choices:
                    print("\nUsage:")
                    print(chunk.usage)
                else:
                    delta = chunk.choices[0].delta
                    # 打印思考过程
                    if hasattr(delta, 'reasoning_content') and delta.reasoning_content is not None:
                        reasoning_content += delta.reasoning_content
                    else:
                        # 开始回复
                        if delta.content != "" and is_answering is False:
                            is_answering = True
                        answer_content += delta.content
            return answer_content

    def process_single(item: Dict) -> Optional[Dict]:
        """处理单个数据项"""
        try:
            context = '--------------'.join(e['text'] for e in item['retrieval_list'])
            prompt = f"{prefix}\n\nQuestion:{item['query']}\n\nContext:\n\n{context}"
            
            return {
                'query': item['query'],
                'prompt': prompt,
                'model_answer': query_model(prompt),
                'gold_answer': item['answer'],
                'question_type': item['question_type']
            }
        except Exception as e:
            print(f"处理失败: {item.get('query','')} - {str(e)}")
            return None

    # 并行处理
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        futures = [executor.submit(process_single, item) for item in process_data]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(process_data)):
            if (res := future.result()) is not None:
                results.append(res)

    # 结果保存逻辑
    output_path = os.path.abspath(output_path)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(results, f)

    return results

In [ ]:
process_qa_pipeline(
    input_path="rerank/dashscope/with_rerank_balance_1_20/retrieval_results_reranked.json",
    output_path="models/qwq_plus/balance_1_20.json",
    model_name="qwq-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    api_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    workers=16,
)

In [ ]:
process_qa_pipeline(
    input_path="rerank/dashscope/with_rerank_balance_3_0_30/retrieval_results_reranked.json",
    output_path="models/gpt_4o_mini/balance_3_0_30.json",
    model_name="gpt-4o-mini-ca",
    api_key=os.getenv("OPENAI_API_KEY_COST"),
    api_url=os.getenv("OPENAI_API_BASE"),
    workers=16,
)

In [ ]:
import os
import json
import time  
from datetime import datetime, timedelta 

def schedule_task(target_hour, target_minute):
    """计算到目标时间的剩余秒数，并添加执行时间校验，白天跑太慢了，就算开多线程也会卡住，只能半夜跑，而且半夜api调用有优惠"""
    now = datetime.now()
    
    # 方法1：直接构造完整时间对象
    target_time = datetime(now.year, now.month, now.day, target_hour, target_minute)
    
    # 处理跨天
    if target_time < now:
        target_time += timedelta(days=1)

    # 计算等待时间
    delta = target_time - now
    wait_seconds = delta.total_seconds()
    
    print(f"当前时间: {now.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"计划执行时间: {target_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"等待 {wait_seconds:.2f} 秒 ({delta})")

    time.sleep(wait_seconds) 

    # 二次校验
    current_after_sleep = datetime.now()
    if current_after_sleep < target_time:
        remaining = (target_time - current_after_sleep).total_seconds()
        time.sleep(remaining)  # 再次正确调用
    
    print(f"实际执行时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 使用示例
if __name__ == "__main__":
    print("程序入口")
    schedule_task(target_hour=1, target_minute=0)
    
    process_qa_pipeline(
        input_path="rerank/dashscope/with_rerank_balance_3_0_30/retrieval_results_reranked.json",
        output_path="models/deepseek_r1/balance_3_0_30.json",
        model_name="deepseek-reasoner",
        api_key=os.getenv("DEEPSEEK_API_KEY"),
        api_url="https://api.deepseek.com",
        workers=100,
    )
        
    
    process_qa_pipeline(
        input_path="rerank/dashscope/with_rerank_balance_1_20/retrieval_results_reranked.json",
        output_path="models/deepseek_r1/balance_1_20.json",
        model_name="deepseek-reasoner",
        api_key=os.getenv("DEEPSEEK_API_KEY"),
        api_url="https://api.deepseek.com",
        workers=100,
    )
    
        
    process_qa_pipeline(
        input_path="rerank/dashscope/with_rerank_balance_3_30/retrieval_results_reranked.json",
        output_path="models/deepseek_r1/balance_3_30.json",
        model_name="deepseek-reasoner",
        api_key=os.getenv("DEEPSEEK_API_KEY"),
        api_url="https://api.deepseek.com",
        workers=100,
    )
    


程序入口
当前时间: 2025-04-18 07:03:51
计划执行时间: 2025-04-19 01:00:00
等待 64568.22 秒 (17:56:08.224139)


In [ ]:
import os
import concurrent.futures

from tqdm import tqdm
from openai import OpenAI
from typing import List, Dict, Optional

prefix = "Below is a question followed by some context from different sources. Please answer the question based on the context. The answer to the question is a word or entity. If the provided information is insufficient to answer the question, respond 'Insufficient Information'. Answer directly without explanation."

def process_qa_pipeline(
    input_path: str,
    output_path: str,
    model_name: str,
    api_key: str,
    api_url: str,
    workers: int = 100,
    
) -> List[Dict]:
    """
    完整的QA处理流水线
    参数:
    input_path: 输入文件路径（JSON格式）
    output_path: 输出文件路径（JSON格式）
    
    返回:
    处理结果列表，每个元素包含：
        - query: 原问题
        - prompt: 生成的完整提示
        - model_answer: 模型回答
        - gold_answer: 正确答案
        - question_type: 问题类型
    """
    with open(input_path, 'r') as f:
        process_data = json.load(f)

    # 初始化OpenAI客户端
    client = OpenAI(api_key=api_key, base_url=api_url)

    def query_model(prompt: str) -> str:
        
        """模型调用统一入口"""
        if "gpt" in model_name.lower() or model_name == "deepseek-reasoner":
            try:
                response = client.chat.completions.create(
                    model=model_name,  # 使用传入的模型名称参数
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0
                )
                return response.choices[0].message.content.strip()
            except Exception as e:
                print(f"API调用失败: {str(e)}")
                return "Error: API调用失败"
            

    def process_single(item: Dict) -> Optional[Dict]:
        """处理单个数据项"""
        try:
            context = '--------------'.join(e['text'] for e in item['retrieval_list'])
            prompt = f"{prefix}\n\nQuestion:{item['query']}\n\nContext:\n\n{context}"
            
            return {
                'query': item['query'],
                'prompt': prompt,
                'model_answer': query_model(prompt),
                'gold_answer': item['answer'],
                'question_type': item['question_type']
            }
        except Exception as e:
            print(f"处理失败: {item.get('query','')} - {str(e)}")
            return None

    # 并行处理
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        futures = [executor.submit(process_single, item) for item in process_data]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(process_data)):
            if (res := future.result()) is not None:
                results.append(res)

    # 结果保存逻辑
    output_path = os.path.abspath(output_path)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(results, f)

    return results